<a href="https://colab.research.google.com/github/AnikaitLakhotia/Machine-Learning/blob/main/CIFAR10_89_37%2C88_12%2C89_12_mnist_99_61(old).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

In [2]:
cifar10 = keras.datasets.cifar10
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

In [3]:
x_valid, x_train = x_train_full[:10000]/255.0 , x_train_full[10000:]/255.0
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]

In [4]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [5]:
width_shift = 3/32
height_shift = 3/32
flip = True

datagen = ImageDataGenerator(
    horizontal_flip=flip,
    width_shift_range=width_shift,
    height_shift_range=height_shift,
    )
datagen.fit(x_train)

In [6]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20,
 restore_best_weights=True)

ReduceOnPlateau = keras.callbacks.ReduceLROnPlateau(
monitor='val_loss',
factor=0.1,
patience=10,
)

root_logdir = os.path.join(os.curdir, "my_logs2")
def get_run_logdir():
 import time
 run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
 return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [7]:
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D,
 kernel_size=3, activation='relu', padding="SAME",kernel_initializer="he_normal",
 kernel_regularizer=keras.regularizers.l2(0.01))
model = keras.models.Sequential([
 DefaultConv2D(filters=64, kernel_size=7, input_shape=[32, 32, 3]),
 layers.BatchNormalization(),
 keras.layers.MaxPooling2D(pool_size=2),
 DefaultConv2D(filters=128),
 layers.BatchNormalization(),
 DefaultConv2D(filters=128),
 layers.BatchNormalization(),
 keras.layers.MaxPooling2D(pool_size=2),
 DefaultConv2D(filters=256),
 layers.BatchNormalization(),
 DefaultConv2D(filters=256),
 layers.BatchNormalization(),
DefaultConv2D(filters=512),
 layers.BatchNormalization(),
 DefaultConv2D(filters=512),
 layers.BatchNormalization(),
 keras.layers.MaxPooling2D(pool_size=2),
 keras.layers.Flatten(),
 keras.layers.Dense(units=128, activation='relu'),
 layers.BatchNormalization(),
 keras.layers.Dropout(0.5),
 keras.layers.Dense(units=64, activation='relu'),
 layers.BatchNormalization(),
 keras.layers.Dropout(0.5),
 keras.layers.Dense(units=10, activation='softmax'),
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
 optimizer="sgd",
 metrics=["accuracy"])

In [ ]:
history = model.fit(x_train,y_train, epochs=1000,
 validation_data=(x_valid, y_valid),
 callbacks=[tensorboard_cb,early_stopping_cb,ReduceOnPlateau])

Epoch 1/1000
1250/1250 [==============================] - 32s 18ms/step - loss: 31.3471 - accuracy: 0.3300 - val_loss: 24.1052 - val_accuracy: 0.4954 - lr: 0.0100
Epoch 2/1000
1250/1250 [==============================] - 21s 17ms/step - loss: 19.3859 - accuracy: 0.4707 - val_loss: 15.4530 - val_accuracy: 0.4297 - lr: 0.0100
Epoch 3/1000
1250/1250 [==============================] - 22s 18ms/step - loss: 12.2742 - accuracy: 0.5413 - val_loss: 9.8747 - val_accuracy: 0.5225 - lr: 0.0100
Epoch 4/1000
 698/1250 [===============>..............] - ETA: 8s - loss: 8.6622 - accuracy: 0.5860

In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
         steps_per_epoch=len(x_train) // 32, epochs=80,validation_data=(x_valid, y_valid),
 callbacks=[tensorboard_cb,early_stopping_cb,ReduceOnPlateau])

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1.5) # set the vertical range to [0-1]
plt.show()